# Broker Manager
In this file we test the basiscs of receiving and sending messages through MQTT connectios

### Old codes

Subscriber

In [ ]:
# -*- coding: utf-8 -*-
"""
----MQTT subscribe code----

Runs on Mosquitto broker
InfluxDB database not included
Ip address available from CMD command: ipconfig all
"""

#!/usr/bin/env python3

# import required packages

import paho.mqtt.client as mqtt
import json
import pandas as pd
#import datetime
import time
from time import sleep
import numpy as np

# defining topics to subscribe

def on_connect(client, userdata, flags, rc):
  print("Connected with result code "+str(rc))
  client.subscribe(topic="activity")

# manipulation of subscribed message data
def on_message(client, userdata, msg):
    # define global variables
    #print(msg.payload) 
    print(str(msg.payload.decode("utf-8")))     # to print the message string
    json_value = json.loads(msg.payload.decode())  #converting message string to python dictionary
    print("time",json_value['time'])   # calling individual data in the json dictionary
    print(json_value)



client = mqtt.Client()
client.connect("127.0.0.1",1883,60,) # verify the IP address before connect

client.on_connect = on_connect
client.on_message = on_message


client.loop_start()
condition = True

while condition:        # the system stays connected as long as the condition is "True"
    sleep(1)
    
client.loop_stop()    
client.disconnect()

Manager

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  5 16:37:07 2019

@author: giova
"""

#!/usr/bin/env python3

import paho.mqtt.client as mqtt
import json
import pandas as pd
#import datetime
import time
from time import sleep
import numpy as np
from influx import InfluxDB

# connect to influx
db = InfluxDB('http://192.168.0.50:8086',precision='ms')
NOME_DATABASE = "lego"

# This is the Subscriber

activity_log = []
states_log = []

count = 0
start_time = 0
th = 0

b3 = 1
b4 = 1
b5 = 0

def on_connect(client, userdata, flags, rc):
  print("Connected with result code "+str(rc))
  client.subscribe("topic/activity")
  client.subscribe("topic/states")
  client.subscribe("topic/buffers")
  client.subscribe("topic/operators")
  

def on_message(client, userdata, msg):
    
    global start_time
    global count
    global th
    global b3
    global b4
    global b5
    global th
    
    if msg.payload.decode() == "start" or msg.payload.decode() == "stop":
        pass
    
    elif msg.topic == "topic/activity":
        
        # print(msg.payload) 
        # print(msg.payload.decode())
        # print(json.loads(msg.payload))
        # print(msg.payload.decode())
        
        json_value = json.loads(msg.payload.decode())  #conversione da stringa a dizionario python
          
        # print("activity",json_value['activity'])
        # print("id",json_value['id'])
        # print("ts",json_value['ts'])
        # print("tag",json_value['tag'])    
        #  json_value['time'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") #change timestamp!
        json_value['ts'] = time.time() #change timestamp!
        activity_log.append(json_value)
        #    influx write senza timestamp
        db.write(NOME_DATABASE,'eventlog', fields={"id":json_value['id'],"type":json_value['tag']}, tags={"activity":json_value['activity']})
    
#        evaluate THROUGHPUT
        if json_value['tag'] =='f' and json_value['activity'] == 1:
            
            try:
                thput = 1/(time.time() - start_time) 
                db.write(NOME_DATABASE,'throughput', fields={"th": thput}, tags={"activity":6})
                start_time = time.time()
            except: 
                pass
            
#            if len(th)<5:
#                th.append(time.time())
#            
#            if len(th)>=5:
#                th = th[1:] +[time.time()] 
#                thput = len(th)/(th[-1] - th[0])
#            
##                thput = count / (time.time() - start_time)
#    #            print("THROUGHPUT: "+str(th))
#                db.write(NOME_DATABASE,'throughput', fields={"th": thput}, tags={"activity":6})

    elif msg.topic == "topic/states":
        
        json_value = json.loads(msg.payload.decode())  #conversione da stringa a dizionario python

#        print("activity",json_value['activity'])
#        print("state",json_value['state'])
        json_value['ts'] = time.time() #change timestamp!
        
        states_log.append(json_value)

#        influx write        
        db.write(NOME_DATABASE,'machine_state', fields={"activity":json_value['activity'], "state":json_value['state']})
    
    
    elif msg.topic == "topic/operators":
        
#        print('message received')
        
        json_value = json.loads(msg.payload.decode())  #conversione da stringa a dizionario python
#        print("BUFFER MESSAGE:"+ str(json_value))
#        MAPPING VALUES FOR DASHBOARD

        if json_value['operator'] == True:
            db.write(NOME_DATABASE,'operators', fields={"pos": json_value['station'] })
    
    
    elif msg.topic == "topic/buffers":
        
        json_value = json.loads(msg.payload.decode())  #conversione da stringa a dizionario python
#        print("BUFFER MESSAGE:"+ str(json_value))
#        MAPPING VALUES FOR DASHBOARD

        
        if json_value['station'] == 3:
        
            if json_value['buffer'] == 'L':
                b3 = 1
                
            if json_value['buffer'] == 'M':
                b3 = 2
                
            if json_value['buffer'] == 'H':
                b3 = 3
                
        if json_value['station'] == 4:
            
            if json_value['buffer'] == 'L':
                b4 = 1
                
            if json_value['buffer'] == 'M':
                b4 = 2
                
            if json_value['buffer'] == 'H':
                b4 = 3
        
        if json_value['station'] == 5:
            
            if json_value['buffer'] == 'NF':
                b5 = 0
                
            if json_value['buffer'] == 'F':
                b5 = 4
                       
#        print("3: " + json_value['3'] + "4: "+str(json_value['4']))
#        print("3: " + str(b3)+ "4: "+str(b4))

#        states_log.append(json_value)

#        influx write        
        db.write(NOME_DATABASE,'buffers', fields={"b3": b3, "b4": b4, "b5": b5})
    
    if msg.payload.decode() == "disc":
        
        print("Disconnecting...")
        client.disconnect()
        
    elif msg.payload.decode() == "start":
        print("STARTED!")
        start_time  = time.time()
    
client = mqtt.Client()
client.connect("192.168.0.50",1883,60)

client.on_connect = on_connect
client.on_message = on_message

th = []

client.loop_start()


condition = True

while condition:
    sleep(1)
    
client.loop_stop()    
client.disconnect()


#DEBUGGING
#json_value = {"time": 123}
#json_value['time'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#activity_log.append(json_value)

df = pd.DataFrame(activity_log)       # pass to pandas df

for a in df.activity.unique().tolist():

    # processing times
    pt = np.array([])
    # tutti gli id che hanno fatto questa attivita
    for i in df.id.unique().tolist():
#        print(i)
        try:
            pt = np.append(pt, (df[(df.activity == a) & (df.id == i) & (df.tag == 'f')].ts.values  - df[(df.activity == a) & (df.id == i) & (df.tag == 's')].ts.values).tolist()  )
        except:
            continue    
    
    np.savetxt('processing'+str(a)+'.txt', pt, delimiter='\n', fmt='%f')
    
    #arrival times:
    # tutti gli id che hanno fatto questa attivita
    #for i in df.id.unique().tolist():
    
    at = np.diff(df[ (df.activity == a) & (df.tag == 'f')].ts)
    np.savetxt('arrivals'+str(a)+'.txt', at, delimiter='\n', fmt='%f')


df = df.sort_values(by ='ts')
df.to_csv(r'log.csv')


# PARTE SU EV3
#activity_log.append({"activity" : 6, "id" : 1, "ts" : 6, "tag" : "f" })
#
#data_out = json.dumps(a, indent = 4)
#client.publish("topic/test", data_out)

### Simply testing of connections

Payload should be in the format:

```json
{
    "time" : 10,
    "workpieces" : 50,
    "free" : "no"
}

```

functions

In [8]:
import paho.mqtt.client as mqtt
from time import sleep
import json

def on_connect(client, userdata, flags, rc):
  print("Connected with result code "+str(rc))
  client.subscribe(topic="activity")

def on_message(client, userdata, msg):
    print(str(msg.payload.decode("utf-8")))     # to print the message string
    json_value = json.loads(msg.payload.decode())  #converting message string to python dictionary
    print("time",json_value['time'])   # calling individual data in the json dictionary
    print(json_value)
    

connection

In [9]:
client = mqtt.Client()
#client.connect("127.0.0.1",1883,60,) # verify the IP address before connect

client.connect("10.169.81.165",1883,60,)

client.on_connect = on_connect
client.on_message = on_message

Loop of connection

In [10]:
client.loop_start()
condition = True

while condition:        # the system stays connected as long as the condition is "True"
    sleep(1)
    
client.loop_stop()    
client.disconnect()

Connected with result code 0
{
    "time" : 11,
    "workpieces" : 50,
    "free" : "no"
}
time 11
{'time': 11, 'workpieces': 50, 'free': 'no'}


KeyboardInterrupt: 

# Test with the Broker Manager

In [9]:
#--- Import modules from the library
from dtwinpylib.dtwinpy.Digital_Twin import Digital_Twin
from dtwinpylib.dtwinpy.broker_manager import Broker_Manager

#--- Reload modules
import importlib
import dtwinpylib
importlib.reload(dtwinpylib.dtwinpy.Digital_Twin)
importlib.reload(dtwinpylib.dtwinpy.broker_manager) 

<module 'dtwinpylib.dtwinpy.broker_manager' from 'c:\\Users\\pedro\\Github Repositories\\Digital_Twin\\Digital Twin\\dtwinpylib\\dtwinpy\\broker_manager.py'>

Normal testing

In [10]:
my_pc_ip_address = "10.169.114.208"
database_path = "databases/broker_test.db"
broker_manager = Broker_Manager(ip_address=my_pc_ip_address, real_database_path= database_path)


In [11]:
broker_manager.run()

.
.
----- Connected with 10.169.114.208 Successfully -----
Subscribed Topics:
|-- 'trace'
|-- 'part_id'
|-- 'RCT-server'
07 March 2023 21:07:54 | Topic: trace | Payload Received: {'machine_id': '1', 'part_id': '12', 'queue_id': '1', 'status': 'Started'}
---- Communication with 10.169.114.208 killed manually----


#### Publishing

In [28]:
broker_manager.publishing(1,12,3)

[BROKER] 07 March 2023 19:20:11 | Topic: RCT-server | Payload Published: {"machine_id": 1, "part_id": 12, "queue_id": 3}


In [15]:
import re

string = 'Part 12 of 1 of 10 ...'
number = re.findall(r'\d+', string)[0]
print(number)  # Output: 2241


12
